In [ ]:
import mlflow
import mlflow.tensorflow
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Sequential
import numpy as np
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import Model, ManagedOnlineEndpoint, ManagedOnlineDeployment
from azure.ai.ml.constants import AssetTypes
import os
import uuid
import requests
import json
# Enable automatic logging
mlflow.tensorflow.autolog()

2024/11/14 18:37:57 WARNING mlflow.utils.autologging_utils: MLflow tensorflow autologging is known to be compatible with 2.7.4 <= tensorflow <= 2.17.0, but the installed version is 2.18.0. If you encounter errors during autologging, try upgrading / downgrading tensorflow to a compatible version, or try upgrading MLflow.


In [2]:
# 1. Generate Dummy Data (Time Series)
# Let's create some sequential data for demonstration.
timesteps = 50   # Number of timesteps for each sample
features = 1     # Number of features
samples = 1000   # Total samples

# Generate sequential data with some noise
x_train = np.random.rand(samples, timesteps, features)
y_train = np.random.rand(samples, 1)  # Target variable

In [3]:
# 2. Define the LSTM Model
model = Sequential([
    Input(shape=(timesteps, features)),  # Specify the input shape here
    LSTM(64),
    Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')

In [4]:
# 3. Start an MLflow Run to Log the Model
with mlflow.start_run() as run:
    # Train the model (logged automatically by mlflow.tensorflow.autolog())
    model.fit(x_train, y_train, epochs=5, batch_size=32, validation_split=0.2)

    # MLflow automatically logs model parameters, metrics, and artifacts
    # including the model, so no need for explicit log_model calls here.

    # Print the run ID for reference (useful for tracking experiments in MLflow UI)
    run_id = run.info.run_id
    print(f"Run ID: {run_id}")

Epoch 1/5
18/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.1652  

25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.1518 - val_loss: 0.0924
Epoch 2/5
17/25 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0967

25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0945 - val_loss: 0.0859
Epoch 3/5
18/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0878

25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0882 - val_loss: 0.0853
Epoch 4/5
18/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0889

25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0889 - val_loss: 0.0848
Epoch 5/5
21/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0929

25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0921 - val_loss: 0.0845
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
Run ID: 9f05cb074437427a821f23c11b597844


In [5]:
# 4. Define Paths and Download the Model
# Define paths for saving model artifacts locally
local_model_path = "./downloaded_model"
model_uri = f"runs:/{run_id}/model"

In [6]:
# Download the model artifacts
mlflow.artifacts.download_artifacts(run_id=run_id, dst_path=local_model_path)
print(f"Model downloaded to {local_model_path}")

/Users/andressaldana/anaconda3/envs/cloud_eng_2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

Model downloaded to ./downloaded_model


In [7]:
# Configure Azure ML client
credential = DefaultAzureCredential()
ml_client = MLClient(
    credential=credential,
    subscription_id="e55be53a-84c8-425d-8430-8151a50b6e6d",
    resource_group_name="TCAML",
    workspace_name="tcaml-workspace",
)

In [8]:
# 6. Register the Model in Azure ML
model_name = "LSTMModel_andres"
model_description = "LSTM model for time series prediction, registered from local MLflow artifacts."

# Register the model using the downloaded artifacts
model = Model(
    path=os.path.join(local_model_path, "model"),  # Path to the downloaded model directory
    type=AssetTypes.MLFLOW_MODEL,                  # Specify the model type as MLflow model
    name=model_name,
    description=model_description,
)

registered_model = ml_client.models.create_or_update(model)
print(f"Model registered with name: {registered_model.name} and version: {registered_model.version}")

Uploading model (0.23 MBs): 100%|██████████| 228316/228316 [00:00<00:00, 335182.71it/s]




Model registered with name: LSTMModel_andres and version: 1


In [9]:
# 7. Create and Deploy an Online Endpoint
# Create a unique name for the endpoint
online_endpoint_name = "lstm-endpoint-" + str(uuid.uuid4())[:8]

# define an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="this is an online endpoint",
    auth_mode="key",
    tags={
        "training_dataset": "random_numbers",
    },
)

In [10]:
# create the online endpoint (2 minutes)
endpoint = ml_client.online_endpoints.begin_create_or_update(endpoint).result()
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)
print(f'Endpoint "{endpoint.name}" with provisioning state "{endpoint.provisioning_state}" is retrieved')

azureml.mlflow could not be imported. Please ensure that latest 'azureml-mlflow' has been installed in the current python environment
Endpoint "lstm-endpoint-cceabdba" with provisioning state "Succeeded" is retrieved


In [11]:
# 8. Define and create a deployment for the model
deployment_name = "lstm-deployment-andres"
# Gretting the registered model from Azure
registered_model_name = model_name
latest_model_version = max([int(m.version) for m in ml_client.models.list(name=registered_model_name)])
model_deploy = ml_client.models.get(name=registered_model_name, version=latest_model_version)

deployment = ManagedOnlineDeployment(
    name=deployment_name,
    endpoint_name=online_endpoint_name,
    model=model_deploy,
    instance_type="Standard_DS2_v2",  # Specify the instance type for deployment # Standard_DS3_v2
    instance_count=1,                 # Number of instances for the deployment
    app_insights_enabled=True  # Enable Application Insights for logging
)

# Deploy the model
ml_client.online_deployments.begin_create_or_update(deployment).result()

Instance type Standard_DS2_v2 may be too small for compute resources. Minimum recommended compute SKU is Standard_DS3_v2 for general purpose endpoints. Learn more about SKUs here: https://learn.microsoft.com/en-us/azure/machine-learning/referencemanaged-online-endpoints-vm-sku-list
Check: endpoint lstm-endpoint-cceabdba exists


azureml.mlflow could not be imported. Please ensure that latest 'azureml-mlflow' has been installed in the current python environment
..........................................................................................................

ManagedOnlineDeployment({'private_network_connection': None, 'package_model': False, 'provisioning_state': 'Succeeded', 'endpoint_name': 'lstm-endpoint-cceabdba', 'type': 'Managed', 'name': 'lstm-deployment-andres', 'description': None, 'tags': {}, 'properties': {'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/e55be53a-84c8-425d-8430-8151a50b6e6d/providers/Microsoft.MachineLearningServices/locations/eastus/mfeOperationsStatus/odidp:6133be2e-2f4f-442b-a4c9-869d74222503:03cb06a0-1887-4109-81d5-fda79e500f08?api-version=2023-04-01-preview'}, 'print_as_yaml': False, 'id': '/subscriptions/e55be53a-84c8-425d-8430-8151a50b6e6d/resourceGroups/TCAML/providers/Microsoft.MachineLearningServices/workspaces/tcaml-workspace/onlineEndpoints/lstm-endpoint-cceabdba/deployments/lstm-deployment-andres', 'Resource__source_path': '', 'base_path': '/Users/andressaldana/Desktop/Reto', 'creation_context': <azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.SystemData object at

In [12]:
# Route 100% of traffic to the deployment
endpoint.traffic = {deployment_name: 100}
ml_client.online_endpoints.begin_create_or_update(endpoint).result()

print(f"Deployment '{deployment_name}' is live at endpoint '{online_endpoint_name}'.")

azureml.mlflow could not be imported. Please ensure that latest 'azureml-mlflow' has been installed in the current python environment
Deployment 'lstm-deployment-andres' is live at endpoint 'lstm-endpoint-cceabdba'.


In [25]:
# 9. Test the Endpoint (Optional)
# Get the endpoint's scoring URI and API key
endpoint = ml_client.online_endpoints.get(online_endpoint_name)
scoring_uri = endpoint.scoring_uri
api_key = ml_client.online_endpoints.get_keys(online_endpoint_name).primary_key

# Original input data: list of sequences, each with 50 timesteps
input_data = [[0.1] * 50]  # Example with one sequence
input_array = np.array(input_data) # Reshape to add the feature dimension
input_array = input_array.reshape((input_array.shape[0], input_array.shape[1], 1)) # New shape: (number of sequences, timesteps, features)

headers = {"Content-Type": "application/json", "Authorization": f"Bearer {api_key}"}
data = json.dumps({"input_data": input_array.tolist()})  # Adjust based on your LSTM input format

# Send a test request
response = requests.post(scoring_uri, headers=headers, data=data)
print("Prediction:", response.json())

Prediction: [[0.228822723031044]]
